# Setup Code
**DO NOT CHANGE THIS!!**
**RUN THE CELL BELOW FIRST!!**

In [8]:
import carla
import random
import os
import time

client_name = os.environ.get("CLIENT_NAME", "DOES NOT EXIST")
if client_name == "DOES NOT EXIST":
    raise Exception("The environment variable for the container name of the carla server has not been set")

# Connect to the client and retrieve the world object
client = carla.Client(client_name, 2000)
world = client.get_world()

# Constants
NPC_VEHICLE_COUNT = 3
SIM_TIME = 60

# Start Adding Code or Markdown Cells Below
Some helpful code cells are provided below that you may choose to use

In [9]:
#### SPAWN VEHICLES ####
"""
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points)))
"""
#### SET ALL VEHICLES TO AUTOPILOT
"""
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)
"""

def save_image(image, image_type="RGB"):
    """Save the image to disk."""
    image_name = f"{image.frame}_{image_type}.png"
    image.save_to_disk(os.path.join('output', image_name))


# Choose a spawn point
spawn_point = random.choice(world.get_map().get_spawn_points())

# Find and spawn a vehicle
vehicle_bp = random.choice(world.get_blueprint_library().filter('vehicle'))
ego_vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Find the blueprint for a camera sensor and configure it
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '800')
camera_bp.set_attribute('image_size_y', '600')
camera_bp.set_attribute('fov', '90')

# Attach the camera to the vehicle
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))  # Adjust as needed for top-of-car view
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego_vehicle)

# Set up the camera to save images to disk
camera.listen(lambda image: save_image(image, "RGB"))

# Spawn NPC vehicles
npc_vehicles = []
for _ in range(NPC_VEHICLE_COUNT):
    spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle_bp = random.choice(world.get_blueprint_library().filter('vehicle'))
    npc_vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)

    if npc_vehicle is not None:
        npc_vehicles.append(npc_vehicle)

# Set all vehicles in world to autopilot
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

# Simulate delay
time.sleep(SIM_TIME)

# Clean up: stop recording and destroy actors
camera.stop()
camera.destroy()
ego_vehicle.destroy()

for npc_vehicle in npc_vehicles:
    npc_vehicle.destroy()

print("Simulation Complete.")


"\nfor vehicle in world.get_actors().filter('*vehicle*'):\n    vehicle.set_autopilot(True)\n"